![ga4](https://www.google-analytics.com/collect?v=2&tid=G-6VDTYWLKX6&cid=1&en=page_view&sid=1&dl=statmike%2Fvertex-ai-mlops%2FApplied+GenAI&dt=Vertex+AI+Matching+Engine+For+Document+Q%26A.ipynb)

# Vertex AI Matching Engine For Document Q&A